In [89]:
import os 

In [92]:
os.chdir("./end-to-end-mlops-project")

In [93]:
%pwd

'd:\\mlops\\end-to-end-mlops-project'

In [94]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
   root_dir: Path
   source_URL: str
   local_data_file: Path
   unzip_dir: Path

In [95]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml , create_directories


In [96]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath =Path('D:\mlops\end-to-end-mlops-project\config\config.yaml'),
        params_filepath = Path("D:\mlops\end-to-end-mlops-project\params.yaml"),
        schema_filepath = Path("D:\mlops\end-to-end-mlops-project\schema.yaml")):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [97]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


In [98]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        logger.info(f'{self.config.source_URL}')
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        logger.info(f'inside the exrtaxt func')
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to: {self.config.unzip_dir}")
        except Exception as e:
            raise e
  


In [99]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-28 15:09:55,546: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\config\config.yaml loaded successfully]
[2024-03-28 15:09:55,548: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\params.yaml loaded successfully]
[2024-03-28 15:09:55,550: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\schema.yaml loaded successfully]
[2024-03-28 15:09:55,551: INFO : common : created directory at: artifacts]
[2024-03-28 15:09:55,552: INFO : common : created directory at: artifacts/data_ingestion]
[2024-03-28 15:09:55,552: INFO : 226428616 : https://github.com/Adi123XD/datasets/raw/main/winequality-red.zip]
[2024-03-28 15:09:56,938: INFO : 226428616 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "15a4573cd527845fc369caf04e55da5b8a23ff3a47c598f96141da62d48e3